<a href="https://colab.research.google.com/github/singhn156/Glove-Helmet-Detection-YoloV4-Darknet/blob/main/Convert_Image_Data_into_YOLO_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#Import Necessary Libraries
import pandas as pd
import os


In [ ]:
# Read csv files containing
os.chdir('/content/drive/My Drive/YOLO_V4/Datasets_and_Annotations/csv_folder')
classes_data = pd.read_csv('class-descriptions-boxable.csv', header=None)
classes_data.head()


,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [ ]:
# Define requried Classes
classes = ['Glove', 'Helmet']


In [ ]:
# Get the strings corresponding to the class name and store it inside a list called as class_strings
class_strings = []
for i in classes:
    # Extract the first column's elements for matching classes Airplane and Parachute
    req_classes = classes_data.loc[classes_data[1] == i]
    string = req_classes.iloc[0][0]
    print(string)
    # Append the strings corresponding to the class name into the list
    class_strings.append(string)
    
print(class_strings)


/m/0174n1
/m/0zvk5
['/m/0174n1', '/m/0zvk5']


In [ ]:

# Get only required columns from annotations csv file and get rid of NOT required columns
annotation_data = pd.read_csv('train-annotations-bbox.csv',
                          usecols=['ImageID',
                                   'LabelName',
                                   'XMin',
                                   'XMax',
                                   'YMin',
                                   'YMax'])
annotation_data.head()


,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [ ]:
# Get only records with matching classes (LabelName = class_strings)
filtered_class_data = annotation_data.loc[annotation_data['LabelName'].isin(class_strings)].copy()
filtered_class_data.head()



,ImageID,LabelName,XMin,XMax,YMin,YMax
83,00001bcc92282a38,/m/0zvk5,0.181875,0.376250,0.085366,0.452158
106,000023aa04ab09ed,/m/0zvk5,0.272727,0.690657,0.000000,0.263462
2165,000134de87dd044b,/m/0zvk5,0.003125,0.091250,0.000000,0.128518
2166,000134de87dd044b,/m/0zvk5,0.200625,0.306250,0.000000,0.120075
2167,000134de87dd044b,/m/0zvk5,0.623125,0.701250,0.365854,0.461538


In [ ]:
# Add new columns that are required for YOLO format
filtered_class_data['classNumber'] = ''
filtered_class_data['center x'] = ''
filtered_class_data['center y'] = ''
filtered_class_data['width'] = ''
filtered_class_data['height'] = ''


In [ ]:
# Iterate through all the class strings and assign a class number according to the order they are appearing in the list
# For e.g.0 to Airplane and 1 to Parachute
for i in range(len(class_strings)):
    # Store the result of this operation into classNumber
    filtered_class_data.loc[filtered_class_data['LabelName'] == class_strings[i], 'classNumber'] = i

# Calculate center x and center y values
filtered_class_data['center x'] = (filtered_class_data['XMax'] + filtered_class_data['XMin']) / 2
filtered_class_data['center y'] = (filtered_class_data['YMax'] + filtered_class_data['YMin']) / 2

# Calculate width and height values
filtered_class_data['width'] = filtered_class_data['XMax'] - filtered_class_data['XMin']
filtered_class_data['height'] = filtered_class_data['YMax'] - filtered_class_data['YMin']


In [ ]:
# Generate the dataframe with YOLO required values 
YOLO_values = filtered_class_data.loc[:, ['ImageID','classNumber','center x','center y','width','height']].copy()
YOLO_values.head()


,ImageID,classNumber,center x,center y,width,height
83,00001bcc92282a38,,,,,
106,000023aa04ab09ed,,,,,
2165,000134de87dd044b,,,,,
2166,000134de87dd044b,,,,,
2167,000134de87dd044b,,,,,
